<a href="https://colab.research.google.com/github/sam505/Machine_Learning/blob/master/QAsystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
# import the necessary libraries
import nltk
import string
import re
import pandas as pd
import json
import inflect
import torch
from google.colab import files
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from transformers import pipeline
from termcolor import  colored
from tqdm import tqdm
import logging
from transformers import DistilBertForQuestionAnswering, BertTokenizerFast, DistilBertTokenizer, BertForQuestionAnswering, AutoTokenizer, AdamW, DistilBertTokenizerFast

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

**Preparing Data**

retrieve data in a dataframe

In [ ]:
def json_to_df(json_file):
    arrayForDF = []
    for current_sub in json_file['data']:
        for current_cntxt in current_sub['paragraphs']:
            cntxt = current_cntxt['context']
            for current_ques in current_cntxt['qas']:
                ques = current_ques['question']
                if(len(ques)>2):
                    is_impossible = current_ques['is_impossible']
                    if(is_impossible == False):
                        for ans in current_ques['answers']:
                            answer_text = ans['text']
                            answer_start = ans['answer_start']
                            record = {
                                "answer": ans,
                                "answer_text" : answer_text,
                                "answer_start": answer_start,
                                "question" : ques,
                                "context" : cntxt,
                            }
                            arrayForDF.append(record)
    df = pd.DataFrame(arrayForDF)
    return df

In [ ]:
with open(r'/content/data.json') as f:  
  data = json.load(f)  

print(data)
json_to_df(data).head(10)

{'data': [{'paragraphs': [{'qas': [{'question': 'What is the main cause of HIV-1 infection in children?', 'id': 262, 'answers': [{'text': 'Mother-to-child transmission (MTCT) is the main cause of HIV-1 infection in children worldwide. ', 'answer_start': 370}], 'is_impossible': False}, {'question': 'What plays the crucial role in the Mother to Child Transmission of HIV-1 and what increases the risk', 'id': 276, 'answers': [{'text': 'DC-SIGNR plays a crucial role in MTCT of HIV-1 and that impaired placental DC-SIGNR expression increases risk of transmission.', 'answer_start': 2003}], 'is_impossible': False}, {'question': 'How many children were infected by HIV-1 in 2008-2009, worldwide?', 'id': 278, 'answers': [{'text': 'more than 400,000 children were infected worldwide, mostly through MTCT and 90% of them lived in sub-Saharan Africa. ', 'answer_start': 2291}], 'is_impossible': False}, {'question': 'What is the role of C-C Motif Chemokine Ligand 3 Like 1 (CCL3L1) in mother to child tran

,answer,answer_text,answer_start,question,context
0,{'text': 'Mother-to-child transmission (MTCT) ...,Mother-to-child transmission (MTCT) is the mai...,370,What is the main cause of HIV-1 infection in c...,Functional Genetic Variants in DC-SIGNR Are As...
1,{'text': 'DC-SIGNR plays a crucial role in MTC...,DC-SIGNR plays a crucial role in MTCT of HIV-1...,2003,What plays the crucial role in the Mother to C...,Functional Genetic Variants in DC-SIGNR Are As...
2,"{'text': 'more than 400,000 children were infe...","more than 400,000 children were infected world...",2291,How many children were infected by HIV-1 in 20...,Functional Genetic Variants in DC-SIGNR Are As...
3,"{'text': 'High copy numbers of CCL3L1, a poten...","High copy numbers of CCL3L1, a potent HIV-1 su...",28143,What is the role of C-C Motif Chemokine Ligand...,Functional Genetic Variants in DC-SIGNR Are As...
4,{'text': 'Dendritic cell-specific ICAM-grabbin...,Dendritic cell-specific ICAM-grabbing non-inte...,3207,What is DC-GENR and where is it expressed?,Functional Genetic Variants in DC-SIGNR Are As...
5,{'text': 'the presence of DC-SIGNR at the plac...,the presence of DC-SIGNR at the placental endo...,28910,How does the presence of DC-SIGNR affect the M...,Functional Genetic Variants in DC-SIGNR Are As...
6,{'text': 'in placenta containing low levels of...,"in placenta containing low levels of DC-SIGNR,...",29090,Why do low levels of DC-SIGNR enhance Mother t...,Functional Genetic Variants in DC-SIGNR Are As...
7,"{'text': 'Without specific interventions, the ...","Without specific interventions, the rate of HI...",2137,What is the percentage of Mother to Child Tran...,Functional Genetic Variants in DC-SIGNR Are As...
8,{'text': 'Genetic variants in CCR5 have been s...,Genetic variants in CCR5 have been shown to in...,27719,Does C-C chemokine receptor type 5 (CCR5) affe...,Functional Genetic Variants in DC-SIGNR Are As...
9,{'text': 'Mannose-binding lectin (MBL) is an i...,Mannose-binding lectin (MBL) is an innate immu...,28335,How does Mannanose Binding Lectin (MBL) affect...,Functional Genetic Variants in DC-SIGNR Are As...


In [ ]:
res = json_to_df(data)
res.to_csv("input.csv")
res

,answer,answer_text,answer_start,question,context
0,{'text': 'Mother-to-child transmission (MTCT) ...,Mother-to-child transmission (MTCT) is the mai...,370,What is the main cause of HIV-1 infection in c...,Functional Genetic Variants in DC-SIGNR Are As...
1,{'text': 'DC-SIGNR plays a crucial role in MTC...,DC-SIGNR plays a crucial role in MTCT of HIV-1...,2003,What plays the crucial role in the Mother to C...,Functional Genetic Variants in DC-SIGNR Are As...
2,"{'text': 'more than 400,000 children were infe...","more than 400,000 children were infected world...",2291,How many children were infected by HIV-1 in 20...,Functional Genetic Variants in DC-SIGNR Are As...
3,"{'text': 'High copy numbers of CCL3L1, a poten...","High copy numbers of CCL3L1, a potent HIV-1 su...",28143,What is the role of C-C Motif Chemokine Ligand...,Functional Genetic Variants in DC-SIGNR Are As...
4,{'text': 'Dendritic cell-specific ICAM-grabbin...,Dendritic cell-specific ICAM-grabbing non-inte...,3207,What is DC-GENR and where is it expressed?,Functional Genetic Variants in DC-SIGNR Are As...
...,...,...,...,...,...
783,{'text': 'at there had been 4000 (95%CI: 1000-...,at there had been 4000 (95%CI: 1000-9700) case...,2927,What was the result of the Imperial College e...,Estimating the Unreported Number of Novel Coro...
784,"{'text': 'y released by the Wuhan Municipal ',...",y released by the Wuhan Municipal,3981,Who release the time series data from 10th to...,Estimating the Unreported Number of Novel Coro...
785,"{'text': ', and later by the National Health C...",", and later by the National Health C",4064,Who released the time series data from after 2...,Estimating the Unreported Number of Novel Coro...
786,"{'text': ', the C i series, as an exponential ...",", the C i series, as an exponential growi",5150,How was the epidemic curve modelled?,Estimating the Unreported Number of Novel Coro...


In [ ]:
def df_to_bert_json(json_file):
  bert_json = []
  for current_sub in json_file['data']:
    for current_cntxt in current_sub['paragraphs']:
      data = {}
      data["context"] = current_cntxt['context']
      for current_ques in current_cntxt['qas']:
          ques = current_ques['question']
          id = current_ques["id"]
          if(len(ques)>2):
              is_impossible = current_ques['is_impossible']
              if(is_impossible == False):
                  for ans in current_ques['answers']:
                      answer_text = ans['text']
                      answer_start = ans['answer_start']
                      data["qas"] = [{"answers": [{"answer_start": answer_start, "text": answer_text}],
                                      "id": id,
                                      "is_possible": False,
                                      "question": ques}]
      bert_json.append(data)
  return bert_json

In [ ]:
with open('data_bert.json', 'w') as f:
    json.dump(df_to_bert_json(data), f)

**Preprocessing**

*Text Lower Case* 

In [ ]:
def text_lowercase(text):
    return text.lower()

*Remove numbers*

In [ ]:
def remove_numbers(text):
    result = re.sub(r'\d+', '', text)
    return result

*We can also convert the numbers into words. This can be done by using the inflect library.*

In [ ]:
p = inflect.engine()
  
# convert number into words
def convert_number(text):
    # split string into list of words
    temp_str = text.split()
    # initialise empty list
    new_string = []
  
    for word in temp_str:
        # if word is a digit, convert the digit
        # to numbers and append into the new_string list
        if word.isdigit():
            temp = p.number_to_words(word)
            new_string.append(temp)
  
        # append the word as it is
        else:
            new_string.append(word)
  
    # join the words of new_string to form a string
    temp_str = ' '.join(new_string)
    return temp_str


*Remove punctuation*

In [ ]:
# remove punctuation
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

*Remove whitespaces*

In [ ]:
# remove whitespace from text
def remove_whitespace(text):
    return  " ".join(text.split())

*Remove default stopwords*

In [ ]:
# remove stopwords function
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return filtered_text

*Stemming:*
Stemming is the process of getting the root form of a word. Stem or root is the part to which inflectional affixes (-ed, -ize, -de, -s, etc.) are added. The stem of a word is created by removing the prefix or suffix of a word. So, stemming a word may not result in actual words.

In [ ]:
stemmer = PorterStemmer()
  
# stem words in the list of tokenised words
def stem_words(text):
    word_tokens = word_tokenize(text)
    stems = [stemmer.stem(word) for word in word_tokens]
    return stems
  

*Lemmatization*:
Like stemming, lemmatization also converts a word to its root form. The only difference is that lemmatization ensures that the root word belongs to the language. We will get valid words if we use lemmatization. In NLTK, we use the WordNetLemmatizer to get the lemmas of words. We also need to provide a context for the lemmatization. 

In [ ]:
lemmatizer = WordNetLemmatizer()
# lemmatize string
def lemmatize_word(word_tokens):
    # provide context i.e. part-of-speech
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in word_tokens]
    return lemmas

# Query Analysis
The question analysis phase consists of three steps: 
1. Question classification. 
2. Tokenization 
3. Identification of Question Focus. 

In [ ]:
input = "How is COVID-19 spread?"
output = text_lowercase(input)
print(output)

how is covid-19 spread?


In [ ]:
output = remove_punctuation(output)
print(output)

how is covid19 spread


In [ ]:
output = remove_whitespace(output)
print(output)

how is covid19 spread


In [ ]:
nltk.download('punkt')
output =  remove_stopwords(output)
output

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['covid19', 'spread']

In [ ]:
nltk.download('wordnet')
output = lemmatize_word(output)
output

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['covid19', 'spread']

**POS  Tagging**

*NN (Noun), VB (Verb), JJ (Adjective), RB(Adverb), IN (Preposition), and CC (Conjunction) and so on.*

In [ ]:
stop_words = set(stopwords.words('english'))
tokenized = sent_tokenize(input)
print(tokenized)
for i in tokenized:
      
    # Word tokenizers is used to find the words 
    # and punctuation in a string
    wordsList = nltk.word_tokenize(i)
  
    # removing stop words from wordList
    wordsList = [w for w in wordsList if not w in stop_words] 
  
    #  Using a Tagger. Which is part-of-speech 
    # tagger or POS-tagger. 
    tagged = nltk.pos_tag(wordsList)
  
    print(tagged)

['How is COVID-19 spread?']
[('How', 'WRB'), ('COVID-19', 'JJ'), ('spread', 'NN'), ('?', '.')]


**Spelling Correction**

**Lemmatization**

In [ ]:
nlp = pipeline('question-answering',model = 'bert-large-cased-whole-word-masking-finetuned-squad')

In [ ]:
query_sample = "How to prevent Corona ?"
relevant_sentence = "When asked why they were wearing masks, several students answered that they were preventing corona"
nlp(question = query_sample, context = relevant_sentence)

{'answer': 'wearing masks',
 'end': 38,
 'score': 0.6434423923492432,
 'start': 25}

In [ ]:
# Python3 program to Check if given words 
# appear together in a list of sentence
  
def check(sentence, words):
    res = [all([k in s for k in words]) for s in sentence]
    return [sentence[i] for i in range(0, len(res)) if res[i]]

In [ ]:
questions =[]
for q in res.question.unique():
    questions.append(q)
questions

['What is the main cause of HIV-1 infection in children?',
 'What plays the crucial role in the Mother to Child Transmission of HIV-1 and what increases the risk',
 'How many children were infected by HIV-1 in 2008-2009, worldwide?',
 'What is the role of C-C Motif Chemokine Ligand 3 Like 1 (CCL3L1) in mother to child transmission of HIV-1?',
 'What is DC-GENR and where is  it expressed?',
 'How does the presence of DC-SIGNR affect the MTCT of HIV-1?',
 'Why do low levels of DC-SIGNR enhance Mother to Child Transmission of HIV-1?',
 'What is the percentage of Mother to Child Transmission of HIV-1, when there is no intervention?',
 'Does C-C chemokine receptor type 5 (CCR5) affect the transmission of HIV-1?',
 'How does Mannanose Binding Lectin (MBL) affect elimination of HIV-1 pathogen?',
 "How can CCR5's effect in HIV-1 transmission be reduced?",
 'What is IFITM?',
 'How many cysteine residues are contained in the first transmembrane domain of IFITM3?',
 'What inhibits S-palmitoylatio

In [ ]:
res.shape

(788, 5)

In [ ]:
len(res.question.unique())

775

In [ ]:
sample_question = res.iloc[400]
sample_question

answer          {'text': 'morbid obesity', 'answer_start': 11203}
answer_text                                        morbid obesity
answer_start                                                11203
question        What risk factor was associated with hospitali...
context         The human viral challenge model: accelerating ...
Name: 400, dtype: object

In [ ]:
def color_answer(question):
    answer_start = question["answer_start"]
    answer_text = question["answer_text"]
    context = question["context"]
    return colored(answer_text,"green")

In [ ]:
print(sample_question["question"])
print()
print("Answer:")
print(color_answer(sample_question))

What risk factor was associated with hospitalization and death during the 2009 H1N1 influence pandemic?

Answer:
morbid obesity


# Train Model


**We train our word embedding models on the context(the texts to which the question relate)**

In [ ]:
# Define and initialize Bert model to be used
modelname = 'deepset/bert-base-cased-squad2'

model = BertForQuestionAnswering.from_pretrained(modelname)
tokenizer = AutoTokenizer.from_pretrained(modelname)

In [ ]:
# Initialize the pipeline with the appropriate tokenizer and model
nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)

## Test the pretrained model

In [ ]:
print("Context: ", res.context[0])

Context:  Functional Genetic Variants in DC-SIGNR Are Associated with Mother-to-Child Transmission of HIV-1

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2752805/

Boily-Larouche, Geneviève; Iscache, Anne-Laure; Zijenah, Lynn S.; Humphrey, Jean H.; Mouland, Andrew J.; Ward, Brian J.; Roger, Michel
2009-10-07
DOI:10.1371/journal.pone.0007211
License:cc-by

Abstract: BACKGROUND: Mother-to-child transmission (MTCT) is the main cause of HIV-1 infection in children worldwide. Given that the C-type lectin receptor, dendritic cell-specific ICAM-grabbing non-integrin-related (DC-SIGNR, also known as CD209L or liver/lymph node–specific ICAM-grabbing non-integrin (L-SIGN)), can interact with pathogens including HIV-1 and is expressed at the maternal-fetal interface, we hypothesized that it could influence MTCT of HIV-1. METHODS AND FINDINGS: To investigate the potential role of DC-SIGNR in MTCT of HIV-1, we carried out a genetic association study of DC-SIGNR in a well-characterized cohort of 197

In [ ]:
print("Question: ", res.question[100])

Question:  What was the last step in the analysis?


In [ ]:
print("Answer: ", res.answer_text[100])

Answer:  amplification step


In [ ]:
# Test the pretrained model
context = res.context[100]
nlp({
    'question': res.question[100],
    'context': context
})

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


{'answer': 'analysis of the EBC demonstrated an InfB infection',
 'end': 11830,
 'score': 0.16873717308044434,
 'start': 11780}

## Creating a dataset

In [ ]:
contexts = list(res.context)
questions = list(res.question)
answers = list(res.answer)

In [ ]:
print(len(contexts), len(questions), len(answers))

788 788 788


In [ ]:
# Split the dataset to train set 80%
train_contexts, val_contexts, train_questions, val_questions, train_answers, val_answers = train_test_split(contexts, questions, answers, test_size=.3)

print(len(train_contexts), len(train_questions), len(train_answers))

551 551 551


In [ ]:
# Add answer_end to the train and validation answers
def add_answer_end(answers, contexts):
  for answer, context in zip(answers, contexts):
    gold_text = answer["text"]
    start_idx = answer["answer_start"]
    end_idx = start_idx + len(gold_text)

    if context[start_idx:end_idx] == gold_text:
      answer["answer_end"] = end_idx

    else:
      for n in [1,2]:
        if context[start_idx-n:end_idx-n] == gold_text:
          answer["answer_start"] = start_idx - n
          answer["answer_end"] = end_idx - n

add_answer_end(train_answers, train_contexts)
add_answer_end(val_answers, val_contexts)

In [ ]:
train_answers

[{'answer_end': 10300,
  'answer_start': 9970,
  'text': ' patients infected with 2019-nCoV may harbour the virus in the intestine at the early or late stage of disease. It is also worth to note none of the patients with viremia blood had positive swabs. These patients would likely be considered as 2019-nCoV negative through routine surveillance, and thus pose a threat to other people.'},
 {'answer_end': 7053,
  'answer_start': 6995,
  'text': 'respiratory bronchioles, alveolar ducts, and alveolar sacs'},
 {'answer_end': 1339, 'answer_start': 1324, 'text': '27 January 2020'},
 {'answer_end': 19426,
  'answer_start': 19332,
  'text': 'may also cause the virus to linger longer in the airway due to the lack of antiviral clearance'},
 {'answer_end': 9379,
  'answer_start': 9168,
  'text': 'to provide further understanding of viral induced exacerbation to identify potential pathways and pathogenesis mechanisms that may be targeted as supplementary care for management and prevention of exace

## Tokenizing

In [ ]:
# Initializing the tokenizer to be used
model_name = "distilbert-base-uncased" 
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name, model_max_length=7000)

In [ ]:
# Tokenizing
train_encodings = tokenizer(train_contexts, train_questions, padding=True, truncation=True)
val_encodings = tokenizer(val_contexts, val_questions, padding=True, truncation=True)

In [ ]:
# function to add end_positions to the encodings
def add_token_positions(encodings, answers):
  start_positions = []
  end_positions = []
  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers[i]["answer_start"]))
    end_positions.append(encodings.char_to_token(i, answers[i]["answer_end"]))

    go_back = 1
    while start_positions[-1] is None:
      start_positions[-1] =  encodings.char_to_token(i, answers[i]['answer_start'] - go_back)
      go_back += 1

    # go_back = 1
    # while end_positions[-1] is None:
    #   end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] - go_back)
    #   go_back += 1
    if end_positions[-1] is None:
      end_positions[-1] = tokenizer.model_max_length

  encodings.update({
      "start_positions": start_positions,
      "end_positions": end_positions
  })

In [ ]:
# add end_positions in the encodings
add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

In [ ]:
print(train_encodings["start_positions"])
print(train_encodings["end_positions"])

[495, 501, 472, 496, 503, 503, 450, 496, 496, 504, 488, 490, 500, 491, 498, 498, 500, 492, 500, 280, 309, 493, 495, 503, 504, 487, 494, 500, 494, 499, 499, 492, 487, 500, 496, 493, 500, 498, 500, 500, 490, 492, 495, 500, 122, 499, 496, 489, 497, 501, 486, 494, 503, 502, 492, 239, 499, 494, 503, 499, 494, 496, 235, 500, 496, 495, 233, 487, 498, 500, 501, 486, 492, 498, 502, 496, 502, 494, 482, 495, 491, 493, 490, 503, 43, 487, 495, 486, 484, 491, 504, 496, 498, 177, 488, 494, 494, 500, 501, 502, 496, 299, 500, 500, 486, 502, 498, 502, 227, 495, 493, 484, 165, 501, 499, 499, 501, 423, 499, 495, 498, 492, 497, 494, 300, 496, 499, 488, 495, 498, 498, 492, 503, 494, 503, 487, 498, 490, 495, 488, 372, 495, 108, 493, 498, 490, 493, 495, 212, 502, 489, 458, 492, 497, 502, 286, 493, 496, 505, 493, 491, 503, 151, 500, 489, 497, 493, 496, 496, 171, 492, 492, 491, 498, 497, 495, 493, 490, 212, 487, 367, 489, 300, 492, 499, 492, 502, 492, 504, 498, 497, 459, 479, 502, 478, 496, 491, 496, 497, 250, 

In [ ]:
train_encodings["start_positions"].count(None)

0

## Transform into a PyTorch dataset

In [ ]:
# class to create datasets from encodings
class QADataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings

  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

  def __len__(self):
    return len(self.encodings.input_ids)

In [ ]:
train_dataset = QADataset(train_encodings)
val_dataset = QADataset(val_encodings)

## Fine tuning the pretrained model

In [ ]:
model = DistilBertForQuestionAnswering.from_pretrained(model_name)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
model.train()
optim = AdamW(model.parameters(), lr=5e-5)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask', 'start_positions', 'end_positions'])

In [ ]:
for epoch in range(5):
  loop = tqdm(train_loader)
  for batch in loop:
    optim.zero_grad()

    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    start_positions = batch["start_positions"].to(device)
    end_positions = batch["end_positions"].to(device)

    outputs = model(input_ids, 
                    attention_mask=attention_mask, 
                    start_positions = start_positions,
                    end_positions = end_positions)
    
    loss = outputs[0]
    loss.backward()
    optim.step()
    loop.set_description(f"Epoch {epoch}")
    loop.set_postfix(loss = loss.item())

Epoch 4: 100%|██████████| 35/35 [00:49<00:00,  1.40s/it, loss=0.49]


In [ ]:
model.eval()

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            

In [ ]:
model_path = "model/custom_distil_bert"

model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('model/custom_distil_bert/tokenizer_config.json',
 'model/custom_distil_bert/special_tokens_map.json',
 'model/custom_distil_bert/vocab.txt',
 'model/custom_distil_bert/added_tokens.json',
 'model/custom_distil_bert/tokenizer.json')

In [ ]:
model = DistilBertForQuestionAnswering.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)

In [ ]:
context = val_contexts[16]
question = val_questions[16]
print(question)
print(val_answers[16])
print(context)
print(nlp({
    "question" : question,
     "context" : context
}))

What was mortality in  South Korea from MERS disease?
{'text': 'ranged from 7 % among younger age groups to 40 % among those aged 60 years and above', 'answer_start': 8955, 'answer_end': 9039}
MERS coronavirus: diagnostics, epidemiology and transmission

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4687373/

SHA: f6fcf1a99cbd073c5821d1c4ffa3f2c6daf8ae29

Authors: Mackay, Ian M.; Arden, Katherine E.
Date: 2015-12-22
DOI: 10.1186/s12985-015-0439-5
License: cc-by

Abstract: The first known cases of Middle East respiratory syndrome (MERS), associated with infection by a novel coronavirus (CoV), occurred in 2012 in Jordan but were reported retrospectively. The case first to be publicly reported was from Jeddah, in the Kingdom of Saudi Arabia (KSA). Since then, MERS-CoV sequences have been found in a bat and in many dromedary camels (DC). MERS-CoV is enzootic in DC across the Arabian Peninsula and in parts of Africa, causing mild upper respiratory tract illness in its camel reservoir and spo

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


{'score': 0.02347426302731037, 'start': 70843, 'end': 70846, 'answer': 'The'}
